In [1]:
import pyomo.environ as pyo

## Enter your data here

In [2]:
# parameters

csc = 0.6
dmg_add = 0.5 + 3.154 + 0.24 + 0.495 + 0.24 + 0.784 + 0.396
dmg_bleed = 0.313
dmg_bers_p = 1.484            # damage while berserking from paragon
dmg_vuln_p = 0.396            # vulnerable damage from paragon
dmg_csd_i = 0.175+0.175+0.35  # csd from weapon implicits
paingorger_roll = 2
paingorger_as = 0.145
paingorger_csc = 0.116
# enter your strength from all sources but rings and amulet
str_no_jewelery = 1811
# all attack speed bonuses from non-gear bonuses
as_non_gear = 0.3 + 0.2 + 0.16 + 0.4
# critical strike chance from base and stats
csc_non_gear = 0.05 + 0.1

# passive skill ranks in talents for Heavy Handed (hh), Cut to the Bone (cttb), Counteroffensive (co), Pit Fighter (pf)
ranks_hh = 3
ranks_cttb = 3
ranks_co = 3
ranks_pf = 3

# maximal number of skill rank affixes on the amulet. Set to 1 if you are not RMT'ing
amu_max_skillrank_affixes = 1
# set to 1 if you want to use cooldown reduction on the amulet, otherwise 0
amu_use_cdr = 1

## Constants

In [3]:
# data for all the affix values, assuming max roll and fully masterworked (1.45), but no mw crits

temper_csd = 0.85 * 1.45
temper_vd = 0.55 * 1.45
temper_bd = 0.55 * 1.45
temper_bash_cleave = 1.05 * 1.45

gear_csd = 0.5 * 1.45
gear_vd = 0.4 * 1.45
gear_csc = 0.06 * 1.45
gear_as = 0.09 * 1.45
gear_str = 90

amulet_csc = 0.08 * 1.45
amulet_as = 0.09 * 1.45
amulet_str_perc = 0.073 * 1.45
amulet_skill_ranks = 3           # 2 skill ranks upgrade to 3 without mw crit but full mw

gem_csd = 0.25
gem_vd = 0.2
gem_basicd = 0.45

## Model starts here

In [4]:
model = pyo.ConcreteModel()

In [5]:
# variable declaration, vd=vulnerable damage, bd=berserking damage, csd=critical strike damage
model.x_vd = pyo.Var(domain=pyo.NonNegativeReals)
model.x_csd = pyo.Var(domain=pyo.NonNegativeReals)
model.x_bd = pyo.Var(domain=pyo.NonNegativeReals)

# introduce an auxiliary variable for berserking damage that caps at 3 for the purpose of calculating Blood Rage multiplier
model.x_bd_capped = pyo.Var(domain=pyo.NonNegativeReals)

In [6]:
# combine parameters with variables so the final damage formula is less verbose
pg = 1 + paingorger_roll
hh = 1 + 0.05*ranks_hh
cttb = 1 + 0.05*ranks_cttb
co = 1 + 0.04*ranks_co
pf = 1 + 0.03*ranks_pf

vd = dmg_vuln_p + model.x_vd
csd = dmg_csd_i + model.x_csd
bd = dmg_bers_p + model.x_bd
bd_capped = dmg_bers_p + model.x_bd_capped
# add includes all additive damage multipliers that are not vd, csd, bd and bleeding damage. add_b includes bleeding damage
add = dmg_add
add_bld = add + dmg_bleed

ats = paingorger_as + as_non_gear
csc = 0.6

str_multi = 1 + (str_no_jewelery)*0.1

In [7]:
# subexpressions for the objective function

# subexpression for the hit damage part of bash, assuming Adaptability and Moonrise(1.8) do not interact with Berserk Ripping
# assuming vulnerable damage works with Paingorger, but crit does not
# 1.15 Two-Handed Mace Expertise
bash_hit = 1.8 * (1+bd_capped*0.1) * ((1-csc+pg)*(1+add+vd+bd) + csc*1.5*hh*1.15*(1+add+vd+csd+bd))

# subexpression for the bleed caused by Berserk Ripping
# multipliers that apply to both normal bleeds and gushing wounds bleeds already included in the base
br_base = (1+pg) * 0.6 * (1+vd*0.15) * (1+bd_capped*0.1) * (1+add_bld+vd+bd) * cttb
bash_bleed = (1-csc)*br_base + csc*br_base*(1+csd)*(1+0.5*1.4)*hh*1.15

In [8]:
# preliminary stat budget of 500+500+150
stat_budget = 11.5

In [9]:
# objective function maximizes the total multiplier incl. attack speed, assuming enemies are always vulnerable (1.2 multiplier irrelevant for the optimization)
total_bash_multi = bash_hit+bash_bleed
model.obj = pyo.Objective(expr = total_bash_multi, sense=pyo.maximize)

# constraints for the stat budget
model.statbudget = pyo.Constraint(expr = model.x_vd+model.x_csd+model.x_bd <= stat_budget)
# limit the capped berserking damage variable to the uncapped one, and limit it to 3
model.bd1 = pyo.Constraint(expr = bd_capped <= bd)
model.bd2 = pyo.Constraint(expr = bd_capped <= 3)

In [10]:
model.pprint()

4 Var Declarations
    x_bd : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeReals
    x_bd_capped : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeReals
    x_csd : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeReals
    x_vd : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeReals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 0.4*(2.4*(1 + (0.396 + x_vd)*0.15)*(1 + (1.484 + x_bd_capped)*0.1)*(0.396 + x_vd + 7.122 + 1.484 + x_bd)*1.15) + 0.6*(2.4*(1 + (0.396 + x_vd)*0.15)*(1 + (1.484 + x_bd_c

In [11]:
opt = pyo.SolverFactory('couenne', executable = 'C:/Users/Daniel/idaes_opti_venv/solvers/couenne.exe')
results = opt.solve(model, tee=True)

Couenne 0.5.8 -- an Open-Source solver for Mixed Integer Nonlinear Optimization
Mailing list: couenne@list.coin-or.org
Instructions: http://www.coin-or.org/Couenne
couenne: 
ANALYSIS TEST: Couenne: new cutoff value -6.6278802372e+02 (0.003 seconds)
NLP0012I 
              Num      Status      Obj             It       time                 Location
NLP0014I             1         OPT -1130.0997       11 0.001
Couenne: new cutoff value -1.1300996637e+03 (0.006 seconds)
Loaded instance "C:\Users\Daniel\AppData\Local\Temp\tmpeyz54gno.pyomo.nl"
Constraints:            3
Variables:              4 (0 integer)
Auxiliaries:           10 (0 integer)

Coin0506I Presolve 20 (-4) rows, 10 (-4) columns and 65 (-5) elements
Clp0006I 0  Obj -6195.1082 Primal inf 1566.5447 (5)
Clp0006I 12  Obj -2362.1206
Clp0000I Optimal - objective value -2362.1206
Clp0032I Optimal objective -2362.120559 - 12 iterations time 0.002, Presolve 0.00
Clp0000I Optimal - objective value -2362.1206
Cbc0012I Integer solution of 

In [12]:
if (results.solver.status == pyo.SolverStatus.ok) and (results.solver.termination_condition == pyo.TerminationCondition.optimal):
    print('Found optimal solution, with the following variable values:')
else:
    print('No optimal solution found, but these are the best variable assignments:')

total_vd = model.x_vd.value+dmg_vuln_p
total_csd = model.x_csd.value+dmg_csd_i
total_bd = model.x_bd.value+dmg_bers_p

print('Total Vulnerable Damage:', total_vd)
print('Total (raw) Critical Strike Damage:', total_csd)
print('Total Berserking Damage:', total_bd)

print('Ratio of CSD to VD:', total_csd/total_vd)
print('Objective function value: ', pyo.value(model.obj))

Found optimal solution, with the following variable values:
Total Vulnerable Damage: 5.780791986272447
Total (raw) Critical Strike Damage: 5.299208123585443
Total Berserking Damage: 3.000000005141998
Ratio of CSD to VD: 0.9166924075748423
Objective function value:  1130.0996636836742


In [123]:
print(model.x_csd.value)

4.599208123585443


In [124]:
print(model.x_vd.value)

5.384791986272447


In [125]:
print(model.x_bd.value)

1.516000005141998


In [126]:
print(pyo.value(model.obj))

1130.0996636836742


In [111]:
print(model.x_bd_capped.value)

1.5159999999999996
